In [4]:
# Імпортуємо необхідні бібліотеки
import pandas as pd
from surprise import Dataset, Reader, KNNBasic, SVD
from surprise.model_selection import train_test_split, cross_validate

# Завантаження датасету ML-100K
data = Dataset.load_builtin('ml-100k')

# Отримуємо дані у форматі DataFrame
df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])

# Виведення перших 5 рядків завантаженого датасету
print("Перші 5 рядків датасету:")
print(df.head())

# Розподіл даних на навчальну та тестову вибірки
trainset, testset = train_test_split(data, test_size=0.2)

# Ініціалізація алгоритмів
algo_knn = KNNBasic()
algo_svd = SVD()

# Тренування алгоритмів
algo_knn.fit(trainset)
algo_svd.fit(trainset)

# Крос-валідація для KNN
knn_results = cross_validate(algo_knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Крос-валідація для SVD
svd_results = cross_validate(algo_svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Обчислення середнього MAE
mae_knn = knn_results['test_mae'].mean()
mae_svd = svd_results['test_mae'].mean()

print(f'MAE KNN: {mae_knn}, MAE SVD: {mae_svd}')

# Обрання найкращого алгоритму на основі MAE
best_algorithm = 'KNN' if mae_knn < mae_svd else 'SVD'
print(f'Найкращий алгоритм: {best_algorithm}')

# Отримання рекомендацій для користувача з ID 196
user_id = 196
item_ids = df['item_id'].unique()
predictions = []

# Обчислення прогнозованих оцінок
for item_id in item_ids:
    pred = algo_svd.predict(user_id, item_id)
    predictions.append((item_id, pred.est))

# Сортування та вибір 10 фільмів з найвищими оцінками
predictions.sort(key=lambda x: x[1], reverse=True)
top_10_recommendations = predictions[:10]

print("Рекомендації для користувача 196:", top_10_recommendations)

# Створення власної рекомендаційної системи
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)

# Розподіл даних
trainset = data.build_full_trainset()

# Ініціалізація та тренування алгоритму KNN
algo = KNNBasic()
algo.fit(trainset)

# Отримання рекомендацій для користувача 196 у власній системі
predictions = []
for item_id in df['item_id'].unique():
    pred = algo.predict(user_id, item_id)
    predictions.append((item_id, pred.est))

# Сортування та вибір 10 фільмів з найвищими оцінками
predictions.sort(key=lambda x: x[1], reverse=True)
top_recommendations = predictions[:10]

print("Рекомендації для користувача 196 (власна система):", top_recommendations)

# Висновки
print("\nВисновки:")
print("1. Датасет ML-100K успішно завантажено та оброблено.")
print("2. Реалізовано два алгоритми: KNN та SVD.")
print("3. Оцінено алгоритми за допомогою крос-валідації на основі MAE.")
print("4. Отримано рекомендації для конкретного користувача.")
print("5. Побудовано власну рекомендаційну систему.")


Перші 5 рядків датасету:
  user_id item_id  rating  timestamp
0     196     242     3.0  881250949
1     186     302     3.0  891717742
2      22     377     1.0  878887116
3     244      51     2.0  880606923
4     166     346     1.0  886397596
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9799  0.9797  0.9795  0.9747  0.9793  0.9786  0.0020  
MAE (testset)     0.7724  0.7727  0.7750  0.7704  0.7750  0.7731  0.0017  
Fit time          0.53    0.40 